# 10_정수 인코딩(Integer Encoding)

컴퓨터는 텍스트보다는 숫자를 더 잘 처리한다. 그 때문에 자연어 처리에서는 텍스트를 숫자로 바꾸는 여러가지 기법들이 있다. 그리고 그러한 기법들을 본격적으로 적용시키기 위한 첫 단계로 각 단어를 고유한 정수에 맵핑(mapping)시키는 전처리 작업이 필요할 때가 있다. 
<br>

예를 들어 갖고 있는 텍스트가 단어 5,000개로 구성되어 있다면, 5,000개의 단어들 각각에 1번부터 5,000번까지 맵핑되는 고유한 정수, 다른 표현으로 말하자면 인덱스를 부여하는 것이다. 가령, book은 150번, dog는 171번, love는 192번, books는 212번과 같이 숫자를 부여한다고 생각하면 된다. 이러한 인덱스를 부여하는 방법은 여러 가지가 있을 수 있다. 랜덤으로 부여하기도 하지만, 보통은 전처리 또는 빈도수가 높은 단어들만 사용하기 위해 단어 출현 빈도수를 기준으로 정렬한 후에 인덱스를 준다.

## 1. 정수 인코딩(Integer Encoding) 
왜 이러한 작업이 필요한지에 대해서는 뒷부분의 [원 핫 인코딩](), [워드 임베딩]() 챕터 등에서 알아보기로 하고 여기서는 어떤 과정으로 단어에 정수 인덱스를 부여하는지에 대해서만 정리한다. 
<br>

단어에 정수(인덱스)를 부여하는 방법 중에는 우선 단어 출현 빈도 순으로 정렬한 단어 집합(vocabulary)를 만들고, 이 빈도수가 높은 순서대로 1, 2, 3....과 같이 정수 인덱스를 부여하는 방법이 있다. 아래에서는 이해를 돕기 위해 단어의 빈도수가 적당히 분포되도록 구축한 텍스트 데이터를 가지고 실습한다.  

### 1) dictionary 사용하기

In [5]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [6]:
text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

### 여러 문장이 함께 있는 텍스트 데이터 문장 토큰화

In [7]:
# 문장 토큰화 
text = sent_tokenize(text)
print(text)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


기존 텍스트 데이터가 문장 단위로 토큰화되는 것을 확인할 수 있다. 이제 정제 작업을 병행하여, 단어 토큰화를 수행한다. 

In [11]:
# 정제와 단어 토큰화
vocab = {} # 파이썬의 dictionary 자료형
sentences = []
stop_words = set(stopwords.words('english'))

for i in text:
    sentence = word_tokenize(i)
    result = []
    
    
    for word in sentence:
        word = word.lower() # 모든 단어를 소문자로 변경, 단어의 개수를 줄임
        
        if word not in stop_words: # 단어 토큰화된 결과에 대해 불용어를 제거함
            if len(word) > 2: # 단어 길이가 2 이하인 경우 추가 제거함
                result.append(word)
                
                if word not in vocab:
                    vocab[word] = 0 # key 값과 value값 채워넣기
                vocab[word] += 1
    sentences.append(result)
print(sentences)         

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


텍스트를 숫자로 바꾸는 단계라는 것은 본격적인 자연어 처리 작업에 들어간다는 의미이다. 단어가 텍스트일 때만 할 수 있는 최대한의 전처리를 끝내놓아야만 한다. 위의 코드를 보면, 동일한 단어가 대문자로 표기되었다는 이유로 서로 다른 단어로 카운트되는 일이 없도록, 모든 단어를 소문자로 바꿨다. 또한 자연어 처리에서 크게 의미가 없는 불용어와 길이가 짧은 단어를 제거했다. 
<br>

그 결과 현재 vocab에는 중복을 제거한 단어와 각 단어에 대한 빈도수가 기록되어 있다. vocab을 출력해보자.

In [12]:
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


단어가 키(key)로, 단어에 대한 빈도수가 값(value)으로 저장되어 있다. vocab에 단어를 입력하면 빈도수를 리턴한다. 

In [13]:
print(vocab["person"]) # 'person'이라는 단어의 빈도수 출력

3


빈도수가 높은 순서대로 정렬해본다.

In [14]:
vocab_sorted = sorted(vocab.items(), key=lambda x:x[1], reverse=True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


이제 인덱스를 부여합니다. 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여합니다.

In [15]:
word_to_index = {}
i=0
for (word, frequency) in vocab_sorted:
    if frequency > 1: # 빈도수가 적은 단어 제외
        i=i+1
        word_to_index[word] = i
        
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


가장 빈도수가 높은 단어가 1의 인덱스를 가진다. 이러한 작업을 수행하는 동시에 각 단어의 빈도수를 알 경우에만 할 수 있는, 빈도수가 적은 단어를 제외하는 작업을 한다. 등장 빈도가 적은 단어는 자연어 처리에서 큰 의미가 없기 때문이다. 
<br>

자연어 처리를 하다 보면, 텍스트 데이터에 있는 단어를 몯 사용하기보다 빈도수가 가장 높은 n개의 단어만 사용하고 싶은 경우가 많다. 위 단어들은 빈도수가 높은 순으로 낮은 정수가 부여되므로 빈도수가 높은 상위 n개의 단어들만 사용하고 싶다면 vocab에서 정수 인덱스가 1에서 n까지인 단어들만 사용하면 된다. 여기서는 상위 5개 단어만 사용한다.

In [16]:
vocab_size=5 # 상위 5개 단어
words_frequency=[w for w, c in word_to_index.items() if c >= vocab_size +1] # 인덱스가 5 초과인 단어를 제거
for w in words_frequency:
    del word_to_index[w] # 해당 단어에 대한 인덱스 정보 삭제 
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


이제 word_to_index에는 빈도수가 높은 상위 5개의 단어만 저장되었다. 이제 word_to_index를 사용해, 단어 토큰화만 된 상태로 저장된 sentences의 각 단어를 정수로 바꾸는 작업을 진행한다. 
<br>

예를 들어 sentences의 첫 번째 문장은 ['barber', 'person']이었는데, 이 문장에 대해서는 [1,5]로 인코딩한다. 그런데 두 번째 문장인 ['barber', 'good', 'person']에는 word_to_index에 존재하지 않는 단어 'good'이라는 단어가 있다. 
<br>

이처럼 단어 집합에 존재하지 않는 단어들을 Out-Of-Vocabulary(단어 집합에 없는 단어)의 약자로 'OOV'라고 한다. word_to_index에 'OOV'를 새롭게 추가하고, 단어 집합에 없는 단어들은 'OOV'의 인덱스로 인코딩한다. 

In [17]:
word_to_index['OOV'] = len(word_to_index) + 1

이제 word_to_index를 사용해 sentences의 모든 단어들을 인덱스와 맵핑시켜 인코딩하겠습니다. 

In [18]:
encoded = []
for s in sentences:
    temp=[]
    for w in s:
        try:
            temp.append(word_to_index[w])
        except KeyError:
            temp.append(word_to_index['OOV'])
    encoded.append(temp)
print(encoded)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


지금까지 파이썬의 dictionary 자료형으로 정수 인코딩을 진행해봤다. 이보다 더 쉬운 인코딩 방법으로 Counter, FreqDist, enumerate 또는 케라스 토크나이저를 사용할 것을 권장한다.

### 2) 